# HW-02-01: Web Scraping ETF Holdings

In this exercise, you will web scrape the investment holdings from several prominent exchange traded funds (ETFs) and create a pandas DataFrame containing their allocation weights.

Use thw webpage below as an example of what to scrape. You should scrape the ticker symbol (ex: AAPL) and its associated weight. Structure your DataFrame like the one below. Note that the DataFrame should be of type float, so be sure to remove the percent symbols. Also, you may choose whichever ETFs you want

|       | AAPL | NVDA | MSFT | AMZN | AVGO | META | TSLA |
|-------|------|------|------|------|------|------|------|
| QQQ   | 8.47 | 8.68 | 8.02 | 6.11 | 4.67 | 3.38 | 3.87 |
| VOO   | 7.60 | 6.61 | 6.29 | 4.12 | 2.17 | 2.56 | 2.26 |
| VTI   | 6.67 | 5.50 | 5.52 | 3.66 | 1.91 | 2.25 | 1.94 |

QQQ holdings example webpage: https://stockanalysis.com/etf/qqq/holdings/

(NOTE: This website only shows up to the top 50 holdings, so it won't include all holdings in the ETF.)

## Directions
- Write code for each section in new cells directly beneath the relevant markdown sections
- When saving your notebook, please name it according to the Following format (modify appropriately for each assignment):
    - Format: HW-01-01_CarSales_\<LastName\>\<FirstInitial\>.ipynb
    - Example: HW-01-01_CarSales_WestG.ipynb

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
from bs4 import BeautifulSoup
import requests


pd.set_option('display.max_columns',100)
pd.set_option('display.max_rows',100)

plt.style.use('dark_background')

In [2]:
# First Step - Importing first Webpage

qqq = 'https://stockanalysis.com/etf/qqq/holdings/'


page = requests.get(qqq)

soup = BeautifulSoup(page.text, 'html')

In [3]:
# Second Step - Inspacting webpage

print(soup)

<!DOCTYPE html>
<html lang="en">
<head>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<link href="../../../_app/immutable/assets/0.C_kzDqif.css" rel="stylesheet"/>
<link href="../../../_app/immutable/assets/SimpleTable.ByOVbpZJ.css" rel="stylesheet"/><title>QQQ Holdings List - Invesco QQQ Trust Series I - Stock Analysis</title><!-- HEAD_svelte-1m45x84_START --><meta content="A list of holdings for QQQ (Invesco QQQ Trust Series I) with details about each stock and its percentage weighting in the ETF." name="description"/><link href="https://stockanalysis.com/etf/qqq/holdings/" rel="canonical"/><meta content="QQQ Holdings List - Invesco QQQ Trust Series I - Stock Analysis" property="og:title"/><meta content="A list of holdings for QQQ (Invesco QQQ Trust Series I) with details about each stock and its percentage weighting in the ETF." property="og:description"/><meta content="https://stockanalysis.com/etf/qqq/holdings/" property="og:url"/><m

In [4]:
soup.find_all('div' , class_='table-wrap svelte-2d4szo')

[<div class="table-wrap svelte-2d4szo"><table class="svelte-2d4szo"><thead><tr class="svelte-2d4szo"><th class="rrpad svelte-2d4szo">No.</th><th class="rlpad svelte-2d4szo">Symbol</th><th class="shr svelte-2d4szo">Name</th><th class="svelte-2d4szo"><!-- HTML_TAG_START --><span class="hidden sm:inline">%</span> Weight<!-- HTML_TAG_END --></th><th class="hide-column-mobile svelte-2d4szo">Shares</th></tr></thead> <tbody><tr class="svelte-2d4szo"><td class="rrpad svelte-2d4szo">1</td><td class="svelte-2d4szo"><!-- HTML_TAG_START --><a href="/stocks/aapl/">AAPL</a><!-- HTML_TAG_END --></td><td class="shr svelte-2d4szo">Apple Inc.</td><td class="svelte-2d4szo">9.01%</td><td class="hide-column-mobile svelte-2d4szo">125,369,225</td> </tr><tr class="svelte-2d4szo"><td class="rrpad svelte-2d4szo">2</td><td class="svelte-2d4szo"><!-- HTML_TAG_START --><a href="/stocks/msft/">MSFT</a><!-- HTML_TAG_END --></td><td class="shr svelte-2d4szo">Microsoft Corporation</td><td class="svelte-2d4szo">7.79%</

In [5]:
soup.find_all('tr')

[<tr class="svelte-2d4szo"><th class="rrpad svelte-2d4szo">No.</th><th class="rlpad svelte-2d4szo">Symbol</th><th class="shr svelte-2d4szo">Name</th><th class="svelte-2d4szo"><!-- HTML_TAG_START --><span class="hidden sm:inline">%</span> Weight<!-- HTML_TAG_END --></th><th class="hide-column-mobile svelte-2d4szo">Shares</th></tr>,
 <tr class="svelte-2d4szo"><td class="rrpad svelte-2d4szo">1</td><td class="svelte-2d4szo"><!-- HTML_TAG_START --><a href="/stocks/aapl/">AAPL</a><!-- HTML_TAG_END --></td><td class="shr svelte-2d4szo">Apple Inc.</td><td class="svelte-2d4szo">9.01%</td><td class="hide-column-mobile svelte-2d4szo">125,369,225</td> </tr>,
 <tr class="svelte-2d4szo"><td class="rrpad svelte-2d4szo">2</td><td class="svelte-2d4szo"><!-- HTML_TAG_START --><a href="/stocks/msft/">MSFT</a><!-- HTML_TAG_END --></td><td class="shr svelte-2d4szo">Microsoft Corporation</td><td class="svelte-2d4szo">7.79%</td><td class="hide-column-mobile svelte-2d4szo">61,664,150</td> </tr>,
 <tr class="s

In [6]:
# Step 3 - Saving The coloum headings 

qqq_titles = soup.find_all('th')

print(qqq_titles)

[<th class="rrpad svelte-2d4szo">No.</th>, <th class="rlpad svelte-2d4szo">Symbol</th>, <th class="shr svelte-2d4szo">Name</th>, <th class="svelte-2d4szo"><!-- HTML_TAG_START --><span class="hidden sm:inline">%</span> Weight<!-- HTML_TAG_END --></th>, <th class="hide-column-mobile svelte-2d4szo">Shares</th>]


In [7]:
# Step 4 - Looping though and getting text for clumns names

qqq_table_titles = [title.text for title in qqq_titles]

print(qqq_table_titles)


['No.', 'Symbol', 'Name', '% Weight', 'Shares']


In [8]:
test_column = soup.find_all('tr' ,class_='svelte-2d4szo')

In [9]:
import pandas as pd

In [10]:
# Step 5 - Converint to a dataframe 


qqq_df = pd.DataFrame(columns = qqq_table_titles)

qqq_df 

,No.,Symbol,Name,% Weight,Shares


In [11]:
#qqq_table = soup.find('table', class_ ='svelte-2d4szo').text

#$print(qqq_table)

In [12]:
# Step 6 
    # Looping through our row data, as we looking we are looking to td 
    # stripping each row from td then it in the list
    # looking at the length 
    # appending the row into the next position

for row in test_column[1:]:
    row_data = row.find_all('td')
    inducidual_row_data = [data.text.strip() for data in row_data]   
    length = len(qqq_df)
    qqq_df.loc[length] = inducidual_row_data

In [13]:
qqq_df


,No.,Symbol,Name,% Weight,Shares
0,1,AAPL,Apple Inc.,9.01%,"125,369,225"
1,2,MSFT,Microsoft Corporation,7.79%,"61,664,150"
2,3,NVDA,NVIDIA Corporation,7.42%,"203,117,790"
3,4,AMZN,"Amazon.com, Inc.",6.31%,"87,210,834"
4,5,AVGO,Broadcom Inc.,4.32%,"64,068,069"
5,6,META,"Meta Platforms, Inc.",3.79%,"18,080,733"
6,7,TSLA,"Tesla, Inc.",3.71%,"30,084,957"
7,8,GOOGL,Alphabet Inc.,3.01%,"48,461,262"
8,9,GOOG,Alphabet Inc.,2.87%,"45,898,420"
9,10,COST,Costco Wholesale Corporation,2.70%,"9,040,911"


In [14]:
# Step 7 Cleand Data

qqq_df_clean = qqq_df.drop(['No.', 'Name', 'Shares'], axis=1)


In [15]:
# New Data Frame
qqq_df_clean['% Weight'] = qqq_df_clean['% Weight'].str.replace('%','')
qqq_df_clean.rename(columns = {'% Weight':'Weight'},inplace =True)
print(qqq_df_clean)

   Symbol Weight
0    AAPL   9.01
1    MSFT   7.79
2    NVDA   7.42
3    AMZN   6.31
4    AVGO   4.32
5    META   3.79
6    TSLA   3.71
7   GOOGL   3.01
8    GOOG   2.87
9    COST   2.70
10   NFLX   2.59
11   TMUS   1.68
12   CSCO   1.50
13    LIN   1.32
14    PEP   1.28
15   ISRG   1.27
16   ADBE   1.20
17   QCOM   1.19
18    AMD   1.17
19   PLTR   1.12
20    TXN   1.05
21   INTU   1.05
22   BKNG   0.97
23   AMGN   0.95
24   AMAT   0.92
25    HON   0.90
26  CMCSA   0.80
27    ADP   0.77
28   SBUX   0.76
29   GILD   0.75
30   PANW   0.75
31   VRTX   0.74
32    APP   0.69
33    ADI   0.65
34   LRCX   0.65
35     MU   0.63
36   KLAC   0.61
37   MRVL   0.61
38   MELI   0.61
39    CEG   0.58
40   CRWD   0.58
41   PYPL   0.55
42   INTC   0.52
43   CDNS   0.51
44   CTAS   0.50
45    MAR   0.50
46   SNPS   0.50
47   MDLZ   0.48
48   FTNT   0.48
49   MSTR   0.47


In [16]:
import numpy as np 
import pandas as pd 

In [17]:
qqq_df_clean.shape

(50, 2)

In [18]:
# Step 8 - Filtering the Symbol on Webpage

soup.find_all('th')

[<th class="rrpad svelte-2d4szo">No.</th>,
 <th class="rlpad svelte-2d4szo">Symbol</th>,
 <th class="shr svelte-2d4szo">Name</th>,
 <th class="svelte-2d4szo"><!-- HTML_TAG_START --><span class="hidden sm:inline">%</span> Weight<!-- HTML_TAG_END --></th>,
 <th class="hide-column-mobile svelte-2d4szo">Shares</th>]

In [19]:
soup.find_all('th')[1:-1]

[<th class="rlpad svelte-2d4szo">Symbol</th>,
 <th class="shr svelte-2d4szo">Name</th>,
 <th class="svelte-2d4szo"><!-- HTML_TAG_START --><span class="hidden sm:inline">%</span> Weight<!-- HTML_TAG_END --></th>]

In [20]:
# Saving the Symbol as ETF name


eft_name_1 = soup.find('h1', class_='pb-0.5 md:pb-0').text

etf = eft_name_1[:3]
print(type(etf))
print(etf)

<class 'str'>
QQQ


In [21]:
etf

'QQQ'

In [22]:
# Step 9 - Adding Column for ETF

qqq_df_clean['ETF'] = etf

In [23]:
qqq_df_clean.columns

Index(['Symbol', 'Weight', 'ETF'], dtype='object')

In [24]:
qqq_df_clean

,Symbol,Weight,ETF
0,AAPL,9.01,QQQ
1,MSFT,7.79,QQQ
2,NVDA,7.42,QQQ
3,AMZN,6.31,QQQ
4,AVGO,4.32,QQQ
5,META,3.79,QQQ
6,TSLA,3.71,QQQ
7,GOOGL,3.01,QQQ
8,GOOG,2.87,QQQ
9,COST,2.70,QQQ


In [25]:
# Step 10 Converting to a Pivot Table

pivot_table = qqq_df_clean.pivot_table(index= ['ETF'], columns =['Symbol'],values=['Weight'])

pivot_table



# For loop for each URL
# Megring


Weight                                                             \
Symbol   AAPL ADBE   ADI   ADP  AMAT   AMD  AMGN  AMZN   APP  AVGO  BKNG   
ETF                                                                        
QQQ      9.01  1.2  0.65  0.77  0.92  1.17  0.95  6.31  0.69  4.32  0.97   

                                                                            \
Symbol  CDNS   CEG CMCSA COST  CRWD CSCO CTAS  FTNT  GILD  GOOG GOOGL  HON   
ETF                                                                          
QQQ     0.51  0.58   0.8  2.7  0.58  1.5  0.5  0.48  0.75  2.87  3.01  0.9   

                                                                               \
Symbol  INTC  INTU  ISRG  KLAC   LIN  LRCX  MAR  MDLZ  MELI  META  MRVL  MSFT   
ETF                                                                             
QQQ     0.52  1.05  1.27  0.61  1.32  0.65  0.5  0.48  0.61  3.79  0.61  7.79   

                                                                               \
Symbol  MSTR    MU  NFLX  NVDA  PANW   PEP  PLTR  PYPL  QCOM  SBUX SNPS  TMUS   
ETF                                                                             
QQQ     0.47  0.63  2.59  7.42  0.75  1.28  1.12  0.55  1.19  0.76  0.5  1.68   

                          
Symbol  TSLA   TXN  VRTX  
ETF                       
QQQ     3.71  1.05  0.74